In [1]:
import os
from glob import glob

import moly
import numpy as np

from HEML.utils.data import mat_pull, get_fe_positions, get_N_positions
from HEML.utils.dictionaries import *
from HEML.utils.visualization import (get_nodes_and_edges_from_pdb,
                                      mat_to_cones, shift_and_rotate)

root = "../../../data/shobhit_md_compressed/"

In [2]:
def get_files(folder): 
    files =  glob(folder + "**/*.dat", recursive=True)
    files = [i for i in files if "distance_matrix" not in i]
    return files

files_center = get_files(root)

In [49]:
def get_fe_positions(file):
    fe_ID, fe_xyz = None, None
    with open(file, "r") as f:
        readfile = f.readlines()

    for j in readfile:
        line = j.split()
        if "ATOM" in line[0] and ("FE" in line[2] or "FE" in line[1]):
            shift = 0
            if "FE" in line[1]:
                shift = -1
            fe_ID = f"{line[6 + shift]}:{line[7 + shift]}:{line[8 + shift]}"
            fe_xyz = [line[6 + shift], line[7 + shift], line[8 + shift]]
            fe_xyz = [float(x) for x in fe_xyz]
            fe_xyz = np.array(fe_xyz)
            break

    return {"id": fe_ID, "xyz": fe_xyz}

def get_N_positions(file, fe_ID, fe_xyz):
    #print(file)
    N_ID, N_ID2, N_ID3, N_ID4 = None, None, None, None
    N1_xyz, N2_xyz, N3_xyz, N4_xyz = None, None, None, None

    with open(file, "r") as f:
        readfile = f.readlines()

    for j in readfile:
        line = j.split()

        shift = 0
        if len(line[0]) > 6:
            shift = -1

        if (
            "ATOM" in line[0]
            and ("NA" in line[2 + shift] and "HM1" in line[3 + shift])
            
        ):
            N_ID = f"{line[4+shift]}:{line[5+shift]}:{line[2+shift]}"
            try:
                N1_xyz = [float(j[31:38]), float(j[38:45]), float(j[46:54])]
            except:
                N1_xyz = [float(line[-5]), float(line[-4]), float(line[-3])]
        if (
            "ATOM" in line[0]
            and ("NB" in line[2 + shift] and "HM1" in line[3 + shift])
            
        ):
            N_ID2 = f"{line[4+shift]}:{line[5+shift]}:{line[2+shift]}"
            try:
                N2_xyz = [float(j[31:38]), float(j[38:45]), float(j[46:54])]
            except:
                N2_xyz = [float(line[-5]), float(line[-4]), float(line[-3])]
        if (
            "ATOM" in line[0]
            and ("NC" in line[2 + shift] and  "HM1" in line[3 + shift])
            
        ):
            N_ID3 = f"{line[4+shift]}:{line[5+shift]}:{line[2+shift]}"
            try:
                N3_xyz = [float(j[31:38]), float(j[38:45]), float(j[46:54])]
            except:
                N3_xyz = [float(line[-5]), float(line[-4]), float(line[-3])]
        if (
            "ATOM" in line[0]
            and ("ND" in line[2 + shift] and "HM1" in line[3 + shift])
            
        ):
            N_ID4 = f"{line[4+shift]}:{line[5+shift]}:{line[2+shift]}"
            try:
                N4_xyz = [float(j[31:38]), float(j[38:45]), float(j[46:54])]
            except:
                N4_xyz = [float(line[-5]), float(line[-4]), float(line[-3])]

    # if there are missing N atoms, find all of the nitrogens and assign them to the closest N atom

    full_N_dict = {}
    if N_ID == None or N_ID2 == None or N_ID3 == None or N_ID4 == None:
        count = 0
        for j in readfile:
            line = j.split()

            shift = 0
            if len(line[0]) > 6:
                shift = -1

            heme_id = fe_ID.split(":")[0]

            if (
                "ATOM" in line[0]
                and ("N" in line[2 + shift] and "HEM" or "HEC" or "HM1" in line[3 + shift])
                and line[4 + shift] == heme_id
            ):
                try:
                    full_N_dict[count] = {
                        "id": f"{line[4+shift]}:{line[5+shift]}:{line[2+shift]}",
                        "xyz": [float(j[31:38]), float(j[38:45]), float(j[46:54])],
                        "distance_to_iron": np.linalg.norm(
                            np.array(
                                [float(j[31:38]), float(j[38:45]), float(j[46:54])]
                            )
                            - np.array(fe_xyz)
                        ),
                    }
                except:
                    full_N_dict[count] = {
                        "id": f"{line[4+shift]}:{line[5+shift]}:{line[2+shift]}",
                        "xyz": [float(line[-5]), float(line[-4]), float(line[-3])],
                        "distance_to_iron": np.linalg.norm(
                            np.array(
                                [float(j[31:38]), float(j[38:45]), float(j[46:54])]
                            )
                            - np.array(fe_xyz)
                        ),
                    }
                count += 1

        # sort the dictionary by distance to iron
        full_N_dict = {
            k: v
            for k, v in sorted(
                full_N_dict.items(), key=lambda item: item[1]["distance_to_iron"]
            )
        }
        # relabel the keys as 0,1,2,3
        print(full_N_dict)
        full_N_dict = {i: full_N_dict[j] for i, j in enumerate(full_N_dict.keys())}
        # get first 4 values of the dictionary
        full_N_dict = dict(list(full_N_dict.items())[0:4])
        # assign the N_IDs
        N_ID = full_N_dict[0]["id"]
        N_ID2 = full_N_dict[1]["id"]
        N_ID3 = full_N_dict[2]["id"]
        N_ID4 = full_N_dict[3]["id"]
        # assign the N_xyz
        N1_xyz = full_N_dict[0]["xyz"]
        N2_xyz = full_N_dict[1]["xyz"]
        N3_xyz = full_N_dict[2]["xyz"]
        N4_xyz = full_N_dict[3]["xyz"]

    assert N_ID != None, "Nitrogens 1 were not found"
    assert N_ID2 != None, "Nitrogens 2 were not found"
    assert N_ID3 != None, "Nitrogens 3 were not found"
    assert N_ID4 != None, "Nitrogens 4 were not found"

    mean_N_xyz = np.mean(np.array([N1_xyz, N2_xyz, N3_xyz, N4_xyz]), axis=0)

    nitrogen_dict = {
        "mean_N_xyz": mean_N_xyz,
        "N_ID1": N_ID,
        "N_ID2": N_ID2,
        "N_ID3": N_ID3,
        "N_ID4": N_ID4,
        "N1_xyz": N1_xyz,
        "N2_xyz": N2_xyz,
        "N3_xyz": N3_xyz,
        "N4_xyz": N4_xyz,
    }

    return nitrogen_dict

fe_dict = get_fe_positions("../../../data/shobhit_md_compressed/GLAVRSQLL_run3_199.pdb")
#print(fe_dict)
N_dict = get_N_positions(
    "../../../data/shobhit_md_compressed/GLAVRSQLL_run3_199.pdb", 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])
#print(N_dict)


{'id': '31.059:32.145:1.00', 'xyz': array([31.059, 32.145,  1.   ])}
../../../data/shobhit_md_compressed/GLAVRSQLL_run3_199.pdb
{'mean_N_xyz': array([40.49175, 31.0725 , 32.06775]), 'N_ID1': '196:39.466:NA', 'N_ID2': '196:39.433:NB', 'N_ID3': '196:41.605:NC', 'N_ID4': '196:41.463:ND', 'N1_xyz': [39.466, 32.35, 33.268], 'N2_xyz': [39.433, 29.52, 32.862], 'N3_xyz': [41.605, 29.81, 30.885], 'N4_xyz': [41.463, 32.61, 31.256]}


In [182]:


# resting data::::
# WT
pdb_wt = "/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb"
pdb_glv = "/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb"
pdb_gla = "/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb"


#pdb_wt = "../../../data/shobhit_md_compressed/WT/compressed_out/1_01_WT-run2-028.pdb"
#pdb_glv = "../../../data/shobhit_md_compressed/GLVRSQL/compressed_out/2_01_GLVRSQL-run2-283.pdb"
#pdb_gla = "../../../data/shobhit_md_compressed/GLAVRSQLL/compressed_out/3_01_GLAVRSQLL-run3-199.pdb"

# 501  ==> run2-592 10%
f_wt2 = '../../../data/shobhit_md_compressed/WT/compressed_out/efield_cox_592_6-prod-100ns-less.dat'
# 1366 ==> run2-776 27%
f_wt4 = '../../../data/shobhit_md_compressed/WT/compressed_out/efield_cox_776_6-prod-100ns-less.dat'
# 1811 ==> run2-925 36%
f_wt5 = '../../../data/shobhit_md_compressed/WT/compressed_out/efield_cox_925_6-prod-100ns-less.dat'
wt_file_list = [f_wt2, f_wt4, f_wt5]
# GLVRSQL
# 703  ==> run2-217 14%
f_glv = "../../../data/shobhit_md_compressed/GLVRSQL/compressed_out/efield_cox_217_6-prod-100ns-less.dat"
# 645  ==> run4-745 44%
f_glv2 = "../../../data/shobhit_md_compressed/GLVRSQL/compressed_out/efield_cox_745_6-prod-100ns-less.dat"
# 2185 ==> run4-593 13%
f_glv3 = "../../../data/shobhit_md_compressed/GLVRSQL/compressed_out/efield_cox_593_6-prod-100ns-less.dat"
glv_file_list = [f_glv, f_glv2, f_glv3]
# GLAVRSQLL
# 554  ==> GLAVRSQLL-run4-504.top 11%
f_gla  = "../../../data/shobhit_md_compressed/GLAVRSQLL/compressed_out/efield_cox_504_6-prod-100ns-less.dat"
# 3467 ==> GLAVRSQLL-run4-986.top 70%
f_gla2 = "../../../data/shobhit_md_compressed/GLAVRSQLL/compressed_out/efield_cox_986_6-prod-100ns-less.dat"
gla_file_list = [f_gla, f_gla2]


In [183]:
from HEML.utils.data import (
    pdb_to_xyz,
    filter_other_by_distance,
    filter_xyz_by_distance,
    filter_by_residue,
    pull_mats_w_label,
)
from HEML.utils.xyz2mol import xyz2AC_vdW
from rdkit import Chem

def connectivity_to_list_of_bonds(connectivity_mat):
    bonds = []
    for i in range(len(connectivity_mat)):
        for j in range(i + 1, len(connectivity_mat)):
            if connectivity_mat[i][j] > 0:
                bonds.append([i, j])
    return bonds


def get_AC(atoms, xyz, covalent_factor=1.4):
    pt = Chem.GetPeriodicTable()
    # xyz to distance matrix
    xyz = np.array(xyz)
    dist_mat = np.linalg.norm(xyz[:, None, :] - xyz[None, :, :], axis=-1)
    num_atoms = xyz.shape[0]

    AC = np.zeros((num_atoms, num_atoms), dtype=int)
    for i in range(num_atoms):
        a_i = atoms[i]
        Rcov_i = pt.GetRcovalent(a_i) * covalent_factor
        for j in range(i + 1, num_atoms):
            a_j = atoms[j]
            Rcov_j = pt.GetRcovalent(a_j) * covalent_factor
            if dist_mat[i, j] <= Rcov_i + Rcov_j:
                AC[i, j] = 1
    return AC


def pdb_to_xyz(file, ret_residues=False):
    with open(file, "r") as f:
        lines = f.readlines()
    xyz = []
    charge = []
    atom = []
    if ret_residues: residues = []
    
    for line in lines:
        if line.startswith("ATOM") or line.startswith("HETATM"):
            #print(line)
            #print(line[32:40], line[40:48], line[48:56])
            xyz.append([float(line[32:40]), float(line[40:48]), float(line[48:56])])
            # xyz.append([float(i) for i in line.split()[6:9]])
            charge.append(float(line.split()[-2]))
            atom.append(atom_int_dict[line.split()[-1]])
        
        if ret_residues:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                residues.append(line[17:21])
    
    if ret_residues:
        return xyz, charge, atom, residues
    
    return xyz, charge, atom

def get_nodes_and_edges_from_pdb(
    file="../../data/pdbs_processed/1a4e.pdb", 
    distance_filter=None, 
    heme_filter=False, 
    filter_connectivity=False,
    center=[130.581, 41.541, 38.350],
):
    
    if heme_filter:
        xyz, charge, atom, residues = pdb_to_xyz(file, ret_residues=True)
        
    else:
        xyz, charge, atom = pdb_to_xyz(file)

    print("xyz to pdb")
    
    """filtered_xyz = filter_xyz_by_distance(
        xyz, center=center, distance=distance_filter
    )
    filtered_atom = filter_other_by_distance(
        xyz, atom, center=center, distance=distance_filter
    )
    print(filtered_atom)"""

    filtered_xyz, filtered_atom = filter_by_residue(xyz, atom, residues, "HM1")
    filtered_xyz_fe, filtered_atom_fe = filter_by_residue(xyz, atom, residues, "FE1")
    filtered_xyz_cb, filtered_atom_cb = filter_by_residue(xyz, atom, residues, "CB1")
    #filtered_xyz_bna, filtered_atom_bna = filter_by_residue(xyz, atom, residues, "BNA")
    filtered_xyz_trp, filtered_atom_trp = filter_by_residue(xyz, atom, residues, "TRP")
    filtered_atom = filtered_atom + filtered_atom_fe + filtered_atom_cb #+ filtered_atom_trp #+ filtered_atom_bna
    filtered_xyz = filtered_xyz + filtered_xyz_fe + filtered_xyz_cb #+ filtered_xyz_trp#+ filtered_xyz_bna
    
    #print(filtered_atom)
    print("distance filter")

    connectivity_mat, rdkit_mol = xyz2AC_vdW(filtered_atom, filtered_xyz)

    connectivity_mat = get_AC(filtered_atom, filtered_xyz, covalent_factor=1.3)
    bonds = connectivity_to_list_of_bonds(connectivity_mat)
    print("connectivity")
    #if filter_connectivity: 
    #    filtered_atom, filtered_xyz, filtered_connectivity_mat, track_indices = connectivity_filter(filtered_atom, filtered_xyz, connectivity_mat, track=26)
    #    bonds = connectivity_to_list_of_bonds(filtered_connectivity_mat)
    
    return filtered_atom, bonds, filtered_xyz


def get_molecule_dict(
        file = "../../../data/pdbs_processed/1a4e.pdb",
        fe_dict=None,
        n_dict=None):
    
    atom_list, bond_list, xyz_list = get_nodes_and_edges_from_pdb(
        file, 
        distance_filter=20.0,
        filter_connectivity=False,
        heme_filter=True,
        center = fe_dict["xyz"])

    if fe_dict == None:
        Fe_pos = [130.581,  41.541,  38.350]
    else: 
        Fe_pos = fe_dict["xyz"]
    
    if n_dict==None:        
        NA_pos = [129.775,  39.761,  38.051]
        NB_pos = [130.581,  41.865,  36.409]
        NC_pos = [131.320,  43.348,  38.639]
        ND_pos = [130.469,  41.267,  40.273]
    else: 
        NA_pos = n_dict["N1_xyz"]
        NB_pos = n_dict["N2_xyz"]
        NC_pos = n_dict["N3_xyz"]
        ND_pos = n_dict["N4_xyz"]
        

    center = np.mean([NA_pos, NB_pos, NC_pos, ND_pos], axis = 0)
    x_axis = np.array(NA_pos) - np.array(center)
    x_axis = x_axis / np.linalg.norm(x_axis)
    y_axis = np.array(NB_pos) - np.array(center)
    y_axis = y_axis / np.linalg.norm(y_axis)
    z_axis = np.cross(y_axis, x_axis)
    z_axis = z_axis / np.linalg.norm(z_axis)

    xyz_list = shift_and_rotate(
        xyz_list, 
        center = center, 
        x_axis = x_axis,
        y_axis = y_axis,
        z_axis = z_axis
    )
    diag_dict = {"Fe": [], "N": []}
    string_element = "\n{}\n\n".format(len(atom_list))
    for i, atom in enumerate(atom_list):
        # check if nitrogen
        if atom == 7:
            diag_dict["N"].append(xyz_list[i])
        if atom == 26: 
            
            diag_dict["Fe"].append(xyz_list[i])
        string_element +="{} {} {} {}\n".format(int_atom_dict[atom], xyz_list[i][0], xyz_list[i][1], xyz_list[i][2])
    dict_input = {"symbols": atom_list, "geometry": xyz_list, "connectivity": bond_list}
    #print(dict_input)
    return string_element, dict_input


def plot_field(
    field_file, 
    molecule_file="../../../data/pdbs_processed/1a4e.pdb",
    fe_dict=None,
    n_dict=None
    ):

    string_element, dict_input = get_molecule_dict(
        molecule_file, 
        fe_dict, 
        n_dict)

    component = mat_to_cones(
        mat_pull(field_file), 
        (1, 21, 21, 21, 3),
        bohr_to_ang_conv=True, 
        vector_scale=5,
        std_mean=False,  
        log1=False, 
        unlog1=False,
        min_max=False,
        cutoff=98)

    fig = moly.Figure()
    molecule = moly.Molecule.from_data(string_element, dtype="string")
    fig.add_molecule("heme", molecule)
    fig.add_trace(component)
    # add cube to show the center of the molecule from -3 to 3
    fig.fig.update_layout(yaxis_range=[-4,4], xaxis_range=[-4,4])

    config = {
    'toImageButtonOptions': {
        'format': 'svg', # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 2000,
        'width': 2000,
        'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
    }
    }
    fig.fig.update_scenes(
        xaxis_visible=False, 
        yaxis_visible=False,
        zaxis_visible=False)
    fig.fig.update(
        layout_showlegend=False)
    camera = dict(
        up=dict(x=0, y=0, z=0.7),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=0.25, y=0.25, z=0.3)
    )
    fig.fig.update_layout(scene_camera=camera, dragmode='orbit')
    #fig.fig.show(config=config)
    fig.fig.write_html("./{}.html".format(field_file.split("/")[-1].split(".")[0]), config=config)

In [184]:
file = pdb_gla
print(file)
print(gla_file_list[0])
fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    gla_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
../../../data/shobhit_md_compressed/GLAVRSQLL/compressed_out/efield_cox_504_6-prod-100ns-less.dat
/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [185]:
file = pdb_gla
print(gla_file_list[0])
fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    gla_file_list[1], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/GLAVRSQLL/compressed_out/efield_cox_504_6-prod-100ns-less.dat
/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [186]:
file = pdb_wt

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    wt_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [187]:
file = pdb_wt

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    wt_file_list[1], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [188]:
file = pdb_wt

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    wt_file_list[2], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [189]:
file = pdb_glv

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    glv_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [190]:
file = pdb_glv

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    glv_file_list[1], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [191]:
file = pdb_glv

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    glv_file_list[2], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

/home/santiagovargas/dev/HEML/data/carbene/925_6-prod-100ns-less.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [211]:
pdb_wt = "../../../data/shobhit_md_compressed/1_02_WT-run5-596.pdb"
pdb_glv = "../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb"
pdb_gla = "../../../data/shobhit_md_compressed/3_01_GLAVRSQLL-run3-199.pdb"


wt_file_list = [
    "../../../data/shobhit_compressed/efield_cox_1_01_WT-run2-028.dat",
    "../../../data/shobhit_compressed/efield_cox_1_02_WT-run5-596.dat",
    "../../../data/shobhit_compressed/efield_cox_1_03_WT-run4-328.dat",
]

glv_file_list = [
    "../../../data/shobhit_compressed/efield_cox_1_01_GLVRSQL-run2-283.dat",
    "../../../data/shobhit_compressed/efield_cox_2_02_GLVRSQL-run5-976.dat",
    "../../../data/shobhit_compressed/efield_cox_2_03_GLVRSQL-run5-074.dat",
]

gla_file_list = [
    "../../../data/shobhit_compressed/efield_cox_3_01_GLAVRSQLL-run3-199.dat",
    "../../../data/shobhit_compressed/efield_cox_3_02_GLAVRSQLL-run1-504.dat",
]

In [201]:
file = pdb_gla
print(file)
print(gla_file_list[0])
fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    gla_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/3_01_GLAVRSQLL-run3-199.pdb
../../../data/shobhit_compressed/efield_cox_3_01_GLAVRSQLL-run3-199.dat
../../../data/shobhit_md_compressed/3_01_GLAVRSQLL-run3-199.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [213]:
file = pdb_gla
print(file)

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    gla_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/3_01_GLAVRSQLL-run3-199.pdb
../../../data/shobhit_md_compressed/3_01_GLAVRSQLL-run3-199.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [203]:
file = pdb_glv
print(file)

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    glv_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb
../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [204]:
file = pdb_glv
print(file)

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    glv_file_list[1], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb
../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [210]:
file = pdb_glv
print(file)

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    glv_file_list[2], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb
../../../data/shobhit_md_compressed/2_02_GLVRSQL-run5-976.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [206]:
file = pdb_wt
print(file)

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    wt_file_list[0], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/1_02_WT-run5-596.pdb
../../../data/shobhit_md_compressed/1_02_WT-run5-596.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)


In [207]:
file = pdb_wt
print(file)

fe_dict = get_fe_positions(file)
N_dict = get_N_positions(
    file, 
    fe_ID=fe_dict['id'], 
    fe_xyz=fe_dict['xyz'])


plot_field(
    wt_file_list[1], 
    molecule_file=file,
    fe_dict=fe_dict,
    n_dict=N_dict
    )

../../../data/shobhit_md_compressed/1_02_WT-run5-596.pdb
../../../data/shobhit_md_compressed/1_02_WT-run5-596.pdb
xyz to pdb
distance filter
connectivity
(21, 21, 21)
(21, 21, 21)
(21, 21, 21)
